In [1]:
import torch
print(f"MPS 장치를 지원하도록 build가 되었는가? {torch.backends.mps.is_built()}")
print(f"MPS 장치가 사용 가능한가? {torch.backends.mps.is_available()}") 

MPS 장치를 지원하도록 build가 되었는가? True
MPS 장치가 사용 가능한가? True


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("pilot_exaone")

LangSmith 추적을 시작합니다.
[프로젝트명]
pilot_exaone


# 그래프 State 정의

In [4]:
from typing import TypedDict, List, Dict
from langchain_core.documents import Document

class GraphState(TypedDict):
    filepath: str  # 원본 파일 경로
    analyzed_files: List  # 분석 완료된 파일 목록
    metadata: List[Dict]  # parsing metadata (api, model, usage)
    elements_from_parser: List[Dict]  # 파싱된 요소


In [5]:
state = GraphState(filepath='/Users/daeunbaek/nuebaek/BOAZ/BOAZ_ADV/Daeun/test_docs')
state

{'filepath': '/Users/daeunbaek/nuebaek/BOAZ/BOAZ_ADV/Daeun/test_docs'}

# DB

In [6]:
from langchain_core.documents import Document
import pickle

def load_docs_from_dicts(path):
    with open(path, "rb") as f:
        doc_dicts = pickle.load(f)
    docs = [Document(page_content=d["page_content"], metadata=d["metadata"]) for d in doc_dicts]
    print(f"✅ 로드 완료: {len(docs)} docs")
    return docs

In [ ]:
# 불러오기
final_docs = load_docs_from_dicts("final_docs_dict.pkl")
final_docs

[Document(metadata={'type': 'text', 'page': 1, 'total_pages': 22, 'source': 'Airway management in neonates and infants European Society of Anaesthesiology and Intensive Care and British Journal of Anaesthesia joint guidelines.pdf', 'id': 5, 'paper_title': 'Airway management in neonates and infants', 'author': 'Takashi Asai, Evelien Cools, Alexandria Cronin, Thomas Engelhardt, John Fiadjoe, Alexander Fuchs, Garcia-Marcinkiewicz, Chloe Heath, Mathias Johansen, Jost Kaufmann, Maren Kleine-Brueggeney, Pete G. Kovatsis, Peter Kranke, Andrea C. Lusardi, Clyde Matava, James Peyton, Thomas Riva, Carolina S. Romero, Britta von Ungern-Sternberg, Francis Veyckemans, Arash Afshari', 'key_words': ['airway management, difﬁcult airway, neonate, paediatric', 'anaesthesia, practice guidelines'], 'header': 'European Society of Anaesthesiology and Intensive Care and British Journal of Anaesthesia joint guidelines', 'summary': 'Nicola Disma et al. collaboratively outline airway management guidelines, endo

- pinecone DB 적재

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
import time
from tqdm import tqdm
from uuid import uuid4
from langchain_pinecone import PineconeVectorStore
import os

openai_api_key = os.environ["OPENAI_API_KEY"]
pinecone_api_key = os.environ["PINECONE_API_KEY"]

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key=pinecone_api_key)

index_name = "pilot-test"

# OpenAI 임베딩 생성
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",  # OpenAI의 임베딩 모델 사용
    api_key=openai_api_key)

# 임베딩 모델 차원 확인 (예: OpenAI의 'text-embedding-ada-002'는 1536차원)
EMBEDDING_DIMENSION = 1536  # OpenAI 임베딩 모델 차원

# 인덱스가 이미 존재하는 경우 생성하지 않고 사용
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name, 
        dimension=EMBEDDING_DIMENSION,  # 임베딩의 차원을 설정 (OpenAI 임베딩의 차원)
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
    # 인덱스가 준비될 때까지 대기
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# 인덱스 가져오기
index = pc.Index(index_name)

# PineconeVectorStore를 사용하여 벡터 스토어 생성
vectorstore = PineconeVectorStore(index=index, embedding=embeddings, text_key="page_content")

# 문서에 UUID 할당
uuids = [str(uuid4()) for _ in range(len(final_docs))]

# 벡터 스토어에 문서 추가 (진행 상황 표시)
for doc, uuid in tqdm(zip(final_docs, uuids), total=len(final_docs), desc="Pinecone에 문서 추가 중"):
    vectorstore.add_documents(documents=[doc], ids=[uuid])

print("문서 저장이 완료되었습니다.")

문서 저장이 완료되었습니다.


### Langgraph

In [41]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import Pinecone as LangchainPinecone
from langchain_openai import OpenAIEmbeddings
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage
from langchain_teddynote import logging
from typing import TypedDict, List, Dict
import os

# LangSmith 프로젝트 설정 (LangGraph 로그 추적)
logging.langsmith("pilot_exaone")

LangSmith 추적을 시작합니다.
[프로젝트명]
pilot_exaone


In [42]:
# 그래프 상태 정의
class GraphState(TypedDict):
    question: str
    documents: List[Document]
    answer: str
    messages: List

- vector db / embedding

In [45]:
# Pinecone 설정
api_key = os.environ.get("PINECONE_API_KEY")
if not api_key:
    raise ValueError("❌ PINECONE_API_KEY 환경변수가 설정되지 않았습니다.")
pc = Pinecone(api_key=api_key)
index_name = "pilot-test"
index = pc.Index(index_name)

# 임베딩 모델 (OpenAI - ada)
embedding_model = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

# 벡터스토어 연결
vectorstore = LangchainPinecone.from_existing_index(
    index_name=index_name,
    embedding=embedding_model,
    text_key="page_content"
)

In [46]:
# Retriever + Reranker 구성
cross_encoder = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
reranker = CrossEncoderReranker(model=cross_encoder, top_n=5)
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
retriever = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=base_retriever
)

# LLM 연결 (Ollama - Exaone)
llm = ChatOllama(model='exaone3.5:7.8b', temperature=0.5)

# 프롬프트 템플릿 (한국어 응답)
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Answer in Korean.

#Context:
{context}

#Question:
{question}

#Answer:"""
)

In [47]:
# 문서 검색 노드
def retrieve_documents(state: GraphState):
    question = state["question"]
    docs = retriever.invoke(question)
    return {**state, "documents": docs}

# LLM 답변 생성 노드 (참고 문서 포함)
def generate_answer(state: GraphState):
    question = state["question"]
    docs = state["documents"]
    context = "\n\n".join([doc.page_content for doc in docs])
    formatted_prompt = prompt.format(context=context, question=question)
    response = llm.invoke(formatted_prompt)
    content = response.content if isinstance(response, AIMessage) else str(response)

    # 참고 문서 정리
    references = []
    for i, doc in enumerate(docs):
        metadata = doc.metadata if hasattr(doc, "metadata") else {}
        source = metadata.get("source", "출처 없음")
        page = metadata.get("page", "페이지 없음")
        references.append(f"[{i+1}] 출처: {source}, 페이지: {page}")

    ref_text = "\n".join(references)
    full_response = f"{content}\n\n📚 참고 문서:\n{ref_text}"

    messages = [("user", question), ("assistant", full_response)]
    return {
        **state,
        "answer": full_response,
        "messages": add_messages(state.get("messages", []), messages)
    }

In [48]:
# ✅ LangGraph 구성
builder = StateGraph(GraphState)
builder.add_node("retrieve", retrieve_documents)
builder.add_node("generate", generate_answer)
builder.set_entry_point("retrieve")
builder.add_edge("retrieve", "generate")
builder.set_finish_point("generate")
graph = builder.compile()

# 예시 실행
if __name__ == "__main__":
    memory = MemorySaver()
    question = "What is the review about direct oral anticoagulants?"
    state = GraphState(question=question, documents=[], answer="", messages=[])
    result = graph.invoke(state)
    print("\n답변:\n", result["answer"])


답변:
 이 리뷰는 주요 정형외과 수술을 받는 환자들에서 직접 경구 항응고제(DOA)의 사용에 초점을 맞추고 있습니다. 특히, 이 리뷰는 다음과 같은 측면들을 다룹니다:

1. **안전성과 효과성**: 아파이크사반, 리바록사반, 다빅트라린 등의 DOA가 정맥 혈전 예방에 효과적이며, 주요 출혈 사건의 위험을 전통적인 항응고제인 에녹사파린과 비교했을 때 크게 증가시키지 않는다는 임상 시험 결과를 강조합니다.

2. **지역 신경 차단 마취와의 병행**: DOA를 복용 중인 환자에서 신경 차단 마취를 진행할 때 발생할 수 있는 출혈 합병증 위험을 인지하고, 적절한 관리 지침의 필요성을 설명합니다.

3. **페리코로믹 관리 프로토콜**: 각 DOA의 약동학적 특성에 기반한 항응고제 중단 및 재개 프로토콜의 중요성을 강조하며, 특히 수술 전 항응고제 중단 후 신경 차단 시술까지의 적절한 대기 시간을 제안합니다.

4. **임상 지침**: 유럽 마취학회(ESA), 미국 마취과학회(ASRA), 스칸디나비아 마취학회(SSAI) 등의 임상 지침을 인용하여 DOA를 복용하는 환자의 마취 관리에 대한 권장 사항을 제공합니다.

요약하자면, 이 리뷰는 DOA의 수술 전후 관리, 특히 지역 마취와의 병행에서의 안전성과 효과적인 관리 방법에 대한 포괄적인 평가를 제공하고 있습니다.

📚 참고 문서:
[1] 출처: 1-s2.0-S0952818016300204-main.pdf, 페이지: 1.0
[2] 출처: 1-s2.0-S0952818016300204-main.pdf, 페이지: 2.0
[3] 출처: 1-s2.0-S0952818016300204-main.pdf, 페이지: 5.0
[4] 출처: 1-s2.0-S0952818016300204-main.pdf, 페이지: 6.0
[5] 출처: 1-s2.0-S0952818016300204-main.pdf, 페이지: 7.0


In [50]:
for i, doc in enumerate(final_docs[:5]):
    print(f"{i}번 문서 타입: {type(doc)}")
    print(f"page_content 존재 여부: {hasattr(doc, 'page_content')}")
    print(f"page_content 내용: {doc.page_content[:100]}...\n")


0번 문서 타입: <class 'langchain_core.documents.base.Document'>
page_content 존재 여부: True
page_content 내용: Nicola Disma, Takashi Asai, Evelien Cools, Alexandria Cronin, Thomas Engelhardt, John Fiadjoe, Alexa...

1번 문서 타입: <class 'langchain_core.documents.base.Document'>
page_content 존재 여부: True
page_content 내용: y management, especially in neonates or infants. It is important to deﬁne the optimal techniques and...

2번 문서 타입: <class 'langchain_core.documents.base.Document'>
page_content 존재 여부: True
page_content 내용: low ‘C’ quality of evidence). In summary, we recommend: 1. Use medical history and physical examinat...

3번 문서 타입: <class 'langchain_core.documents.base.Document'>
page_content 존재 여부: True
page_content 내용: ntinuous positive airway pressure or nasal intermit- tent positive pressure ventilation for postextu...

4번 문서 타입: <class 'langchain_core.documents.base.Document'>
page_content 존재 여부: True
page_content 내용: ren’s Hospital of Philadelphia, Philadelphia, PA, USA (AG-M), 0265-0215 Cop

In [51]:
# 예시 실행
if __name__ == "__main__":
    memory = MemorySaver()
    question = "What is the risk of VTE?"
    state = GraphState(question=question, documents=[], answer="", messages=[])
    result = graph.invoke(state)
    print("\n답변:\n", result["answer"])


답변:
 정맥 혈전증(VTE, Venous Thromboembolism)의 위험은 특히 주요 정형외과 수술, 예를 들어 전신 무릎이나 엉덩이 교체 수술을 받는 환자들 사이에서 매우 높습니다. 이러한 환자들은 수술 전후 혈전 예방을 위해 항응고제를 통상적으로 투여받습니다. 따라서 주요 정형외과 수술을 받는 환자들의 경우 VTE 발생 위험이 크게 증가함을 알 수 있습니다. 이는 고령 환자들에게서 더욱 두드러지며, 고령 환자들은 혈전성 질환의 위험이 높고 항응고제 치료를 받고 있는 경우가 많아 출혈 위험도 함께 증가합니다. 따라서 이러한 환자 집단에서는 특히 주의 깊은 관리가 요구됩니다.

📚 참고 문서:
[1] 출처: 1-s2.0-S0952818016300204-main.pdf, 페이지: 1.0
[2] 출처: 1-s2.0-S0952818016300204-main.pdf, 페이지: 1.0
[3] 출처: 1-s2.0-S0952818016300204-main.pdf, 페이지: 3.0
[4] 출처: 1-s2.0-S0952818016300204-main.pdf, 페이지: 2.0
[5] 출처: 1-s2.0-S0952818016300204-main.pdf, 페이지: 9.0
